In [1]:
%run stdPackages.ipynb

# Read database

In [2]:
from pyDbs import read, readSets, broadcast
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['LoadVariation_E','LoadVariation_H','HourlyVariation'],
        'scalars': ['Scalars']}
db = read.dbFromWB(os.path.join(d['data'],'mGFPH.xlsx'), kwargs)
readSets(db)

1000 hours:

In [3]:
[db.__setitem__(k,adj.rc_pd(db[k], db['h'][0:1000])) for k in db.variableDomains('h', types=['set','variable'])];

Time:

In [4]:
import time
times = {}
t0 = time.time()
# Initialize model:
m = mGFPH.mSimple(db) # initialize using the database 'db' from above.
times['setup'] = time.time()-t0
# Initialize blocks:
m.preSolve()
m.initBlocks()
times['initBlocks'] = time.time()-t0-sum(times.values())
# Compile model:
m.blocks.compileParameters()
times['compileParameters'] = time.time()-t0-sum(times.values())
m.blocks.settingsFromCompiled()
times['settingsFromCompiled'] = time.time()-t0-sum(times.values())
m.blocks.inferGlobalDomains()
times['inferGlobalDomains'] = time.time()-t0-sum(times.values())
m.blocks.getDenseArgs()
times['getDenseArgs'] = time.time()-t0-sum(times.values())
args = m.blocks.lp_args
times['lp_args'] = time.time()-t0-sum(times.values())
sol = optimize.linprog(method = m.method, **args)
times['sol'] = time.time()-t0-sum(times.values())
m.postSolve(sol)
times['postSolve'] = time.time()-t0-sum(times.values())
times

{'setup': 0.11709332466125488,
 'initBlocks': 1.9159445762634277,
 'compileParameters': 1.0341877937316895,
 'settingsFromCompiled': 0.0010001659393310547,
 'inferGlobalDomains': 0.21129441261291504,
 'getDenseArgs': 0.651397705078125,
 'lp_args': 0.385509729385376,
 'sol': 2.093674421310425,
 'postSolve': 0.5073814392089844}